# AIW Cloud Connector Demo

### Step 1: Initialize Client and Create WF Templates

In [65]:
import json
from time import sleep

with open("credentials-intber.json") as f:
    credentials = json.load(f)

print("Credentials succesfully read")

Credentials succesfully read


In [66]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_core_sdk.models import Status

# Initializations
resource_group = "default"
name = "aiw-cc-demo"
client = AICoreV2Client(base_url=credentials.get("base_url"),
                        auth_url=credentials.get("auth_url"),
                        client_id=credentials.get("client_id"),
                        client_secret=credentials.get("client_secret"),
                        resource_group=resource_group)

# Check if resource group is available
resource_group = client.resource_groups.get(resource_group)
print(f"The {resource_group.resource_group_id} resource group is {resource_group.status}")

The default resource group is PROVISIONED


### Step 2: Create repository and application to sync WF templates

In [9]:
# Onboard repository
repository = client.repositories.create(name,
                                        credentials.get("repo_url"),
                                        credentials.get("git_user"),
                                        credentials.get("git_pass")
                                        )
print(repository)

# Create application
application = client.applications.create(revision="main",
                                        path=credentials.get("app_path"),
                                        application_name=name,
                                        repository_name=name
                                        )
print(application)


Message: Repository has been on-boarded.
Id: aiw-cc-demo, Message: Application has been successfully created.


### Step 3: Check if application has synced and scenario created

In [67]:
app_status = client.applications.get_status(name)
print(app_status.health_status)

# Get scenarios
scenarios = client.scenario.query()
demo_scenario_exists = name in [s.name for s in scenarios.resources]
print(f"Scenario {name} exists") if demo_scenario_exists else print(f"Scenario {name} not yet available")

# Get executables
executables = client.executable.query(name)
for e in executables.resources:
    print(f"Executable {e.id} v{e.version_id} found")


Healthy
Scenario aiw-cc-demo exists
Executable aiw-cc-demo v1.0.0 found


In [74]:

# Create execution configuration
config = client.configuration.create(name=name, scenario_id=name, executable_id=name, parameter_bindings={})
print(config)
config_id = config.id

Id: dfa551dc-6ce9-4d6f-b069-d261e743e338, Message: Configuration created


In [75]:
# List config
configs = client.configuration.query(name, [name])
config_ids = {c.name: c.id for c in configs.resources}
print(f"For executable {name} - config_id: {config_ids[name]}")
config_id = config_ids[name]

For executable aiw-cc-demo - config_id: cd2a0502-c406-490a-b469-5f1320b9086f


### Pul Image Secrets

In [70]:
response = client.rest_client.post(
    path="/admin/dockerRegistrySecrets",
    body={
        "name": "cp-test-docker-registry",
        "data": credentials.get("docker_secret")
    }
)
print(response)

{'message': 'Secret exists'}


### Step 4: Start Execution

In [88]:
execution = client.execution.create(config_id, resource_group.resource_group_id)
print(execution)


Id: e69dc86a212e7192, Message: Execution scheduled


In [86]:



for i in range(30):
    e = client.execution.get(execution.id)
    print(f"{e} - {e.status}")
    if e.status != Status.UNKNOWN:
        break
    sleep(2)


Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.UNKNOWN
Execution id: eff614b53db04b91 - Status.RUNNING


In [89]:

logs = client.execution.query_logs(execution_id=execution.id)
print(logs)

Log response messages: 2022-11-04T12:49:51.440561519Z stderr F time="2022-11-04T12:49:51.440Z" level=info msg="Starting Workflow Executor" version=v3.4.0, 2022-11-04T12:49:51.444034411Z stderr F time="2022-11-04T12:49:51.443Z" level=info msg="Using executor retry strategy" Duration=1s Factor=1.6 Jitter=0.5 Steps=5, 2022-11-04T12:49:51.444084126Z stderr F time="2022-11-04T12:49:51.444Z" level=info msg="Executor initialized" deadline="0001-01-01 00:00:00 +0000 UTC" includeScriptOutput=false namespace=rg-b6f59f0a-cd927cab podName=e69dc86a212e7192 template="{\"name\":\"connect\",\"inputs\":{},\"outputs\":{},\"metadata\":{\"labels\":{\"ai.sap.com/component\":\"training\",\"ai.sap.com/resourcePlan\":\"starter\",\"ai.sap.com/tenantId\":\"b27e0789-b0c9-4bbc-9960-317f8af27ce5\",\"executables.ai.sap.com/id\":\"aiw-cc-demo\",\"executions.ai.sap.com/id\":\"e69dc86a212e7192\",\"internal.ai.sap.com/realSubaccountId\":\"b27e0789-b0c9-4bbc-9960-317f8af27ce5\",\"internal.ai.sap.com/subAccount\":\"b27e0

### Cleanup

In [11]:
executions = client.execution.query(scenario_id=name)
execution_ids = [e.id for e in executions.resources]
for id in execution_ids:
    resp = client.execution.delete(id)
    print(resp)

In [ ]:
response = client.repositories.delete(name)
print(response)

response = client.applications.delete(name)
print(application)